In [ ]:
Mysql Innodb Cluster 
=================================

Innodb cluster provides high availability solution for Mysql.

It uses 3 features to acheive this - Mysql shell , Group replication and Mysql Router.

In [ ]:
[root@bosprod01 ~]# ifconfig 
        inet 192.168.153.139  netmask 255.255.255.0  broadcast 192.168.153.255

[root@bosprod02 ~]# ifconfig
        inet 192.168.153.140  netmask 255.255.255.0  broadcast 192.168.153.255

[root@bosprod03 ~]# ifconfig
        inet 192.168.153.141  netmask 255.255.255.0  broadcast 192.168.153.255

[root@bosprod03 ~]# cat /etc/hosts

192.168.153.139 bosprod01
192.168.153.140 bosprod02
192.168.153.141 bosprod03

[root@bosprod01 ~]# systemctl stop firewalld
[root@bosprod01 ~]# 

[root@bosprod03 ~]# mysql_config_editor set -u root -p
Enter password: 
[root@bosprod03 ~]# 

mysql> select @@hostname,@@version;
+------------+-----------+
| bosprod01  | 8.0.30    |
+------------+-----------+
+------------+-----------+
| bosprod02  | 8.0.30    |
+------------+-----------+
+------------+-----------+
| bosprod03  | 8.0.30    |
+------------+-----------+

In [ ]:
Install Mysql shell in all servers
==========================================

[root@bosprod01 ~]# yum install mysql-shell
Package mysql-shell-8.4.0-1.el8.x86_64 is already installed.

[root@bosprod01 ~]# rpm -qa | grep mysql-shell
mysql-shell-8.4.0-1.el8.x86_64
[root@bosprod01 ~]# 

In [ ]:

Pre-Checking Instance Configuration for InnoDB Cluster Usage
===================================================================

[root@bosprod01 ~]# mysqlsh

MySQL  localhost  SQL > \js
Switching to JavaScript mode...

MySQL  localhost  JS > \s

MySQL Shell version 8.4.0
Connection Id:                17
Current schema:               
Current user:                 root@localhost


MySQL  localhost  JS > dba.checkInstanceConfiguration('localhost');

Validating local MySQL instance listening at port 3306 for use in an InnoDB Cluster...

ERROR: New account(s) with proper source address specification to allow remote connection from all
instances must be created to manage the cluster.

Dba.checkInstanceConfiguration: User 'root' can only connect from 'localhost'. (RuntimeError)

 MySQL  localhost  JS > 


 MySQL  localhost  JS > dba.configureInstance();

Configuring local MySQL instance listening at port 3306 for use in an InnoDB Cluster...

1) Create remotely usable account for 'root' with same grants and password
2) Create a new admin account for InnoDB Cluster with minimal required grants
3) Ignore and continue
4) Cancel

Please select an option [1]: 2

Account Name: icadmin
Password for new account: ************
Confirm password: ************

applierWorkerThreads will be set to the default value of 4.

NOTE: Some configuration options need to be fixed:
+----------------------------------------+---------------+----------------+--------------------------------------------------+
| Variable                               | Current Value | Required Value | Note                                             |
+----------------------------------------+---------------+----------------+--------------------------------------------------+
| binlog_transaction_dependency_tracking | COMMIT_ORDER  | WRITESET       | Update the server variable                       |
| enforce_gtid_consistency               | OFF           | ON             | Update read-only variable and restart the server |
| gtid_mode                              | OFF           | ON             | Update read-only variable and restart the server |
| server_id                              | 1             | <unique ID>    | Update read-only variable and restart the server |
+----------------------------------------+---------------+----------------+--------------------------------------------------+

Do you want to perform the required configuration changes? [y/n]: n

Dba.configureInstance: Cancelled (RuntimeError)
 MySQL  localhost  JS > 


[root@bosprod01 ~]# cat /etc/my.cnf
binlog-transaction-dependency-tracking=WRITESET
enforce-gtid-consistency=ON
gtid-mode=ON
server-id=1

[root@bosprod01 ~]# systemctl restart mysqld
[root@bosprod01 ~]# 


 MySQL  localhost  JS > dba.configureInstance();

1) Create remotely usable account for 'root' with same grants and password
2) Create a new admin account for InnoDB Cluster with minimal required grants
3) Ignore and continue
4) Cancel

Please select an option [1]: 2

Account Name: icadmin
Password for new account: ************
Confirm password: ************

Creating user icadmin@%.
Account icadmin@% was successfully created.


 MySQL  localhost  JS > dba.checkInstanceConfiguration('icadmin@localhost');
Please provide the password for 'icadmin@localhost': ************

Checking whether existing tables comply with Group Replication requirements...
No incompatible tables detected

Checking instance configuration...
Instance configuration is compatible with InnoDB cluster

The instance 'bosprod01:3306' is valid for InnoDB Cluster usage.

{
    "status": "ok"
}
 MySQL  localhost  JS > 


Do the same checks and config for bosprod02 and bosprod03 instances 
----------------------------------------------------------------------------

[root@bosprod02 ~]# cat /etc/my.cnf
binlog-transaction-dependency-tracking=WRITESET
enforce-gtid-consistency=ON
gtid-mode=ON
server-id=2
[root@bosprod02 ~]# 

[root@bosprod02 ~]# systemctl restart mysqld
[root@bosprod02 ~]# 

 MySQL  localhost  JS > dba.configureInstance();

This instance reports its own address as bosprod02:3306

1) Create remotely usable account for 'root' with same grants and password
2) Create a new admin account for InnoDB Cluster with minimal required grants
3) Ignore and continue
4) Cancel

Please select an option [1]: 2
Account Name: icadmin
Password for new account: ************
Confirm password: ************

Creating user icadmin@%.
Account icadmin@% was successfully created.


 MySQL  localhost  JS > dba.checkInstanceConfiguration('icadmin@localhost');
Please provide the password for 'icadmin@localhost': ************

Checking whether existing tables comply with Group Replication requirements...
No incompatible tables detected

Checking instance configuration...
Instance configuration is compatible with InnoDB cluster

The instance 'bosprod02:3306' is valid for InnoDB Cluster usage.

{
    "status": "ok"
}
 MySQL  localhost  JS > 


[root@bosprod03 ~]# cat /etc/my.cnf
binlog-transaction-dependency-tracking=WRITESET
enforce-gtid-consistency=ON
gtid-mode=ON
server-id=3
[root@bosprod03 ~]# 

[root@bosprod03 ~]# systemctl restart mysqld
[root@bosprod03 ~]# 


 MySQL  localhost  JS > dba.configureInstance();

1) Create remotely usable account for 'root' with same grants and password
2) Create a new admin account for InnoDB Cluster with minimal required grants
3) Ignore and continue
4) Cancel

Please select an option [1]: 2
Account Name: icadmin
Password for new account: ************
Confirm password: ************

Creating user icadmin@%.
Account icadmin@% was successfully created.

The instance 'bosprod03:3306' is valid for InnoDB Cluster usage.

In [ ]:
Creating an Innodb cluster 
=============================================

Once instances are prepared , create cluster using function dba.CreateCluster().

The instance where you are running this function becomes the seed and other instances becomes the replica which you add to
this cluster later.



 MySQL  localhost  JS > \connect icadmin@localhost

Creating a session to 'icadmin@localhost'
Please provide the password for 'icadmin@localhost': ************
Save password for 'icadmin@localhost'? [Y]es/[N]o/Ne[v]er (default No): y

 MySQL  localhost:33060+ ssl  JS > var cluster = dba.createCluster('bosprodcluster');

A new InnoDB Cluster will be created on instance 'bosprod01:3306'.

Adding Seed Instance...
Cluster successfully created. Use Cluster.addInstance() to add MySQL instances.
At least 3 instances are needed for the cluster to be able to withstand up to
one server failure.

 MySQL  localhost:33060+ ssl  JS > cluster.status();
{
    "clusterName": "bosprodcluster", 
    "defaultReplicaSet": {
        "name": "default", 
        "primary": "bosprod01:3306", 
        "ssl": "REQUIRED", 
        "status": "OK_NO_TOLERANCE", 
        "statusText": "Cluster is NOT tolerant to any failures.", 
        "topology": {
            "bosprod01:3306": {
                "address": "bosprod01:3306", 
                "memberRole": "PRIMARY", 
                "mode": "R/W", 
                "readReplicas": {}, 
                "replicationLag": "applier_queue_applied", 
                "role": "HA", 
                "status": "ONLINE", 
                "version": "8.0.30"
            }
        }, 
        "topologyMode": "Single-Primary"
    }, 
    "groupInformationSourceMember": "bosprod01:3306"
}


Adding instances to an innodb cluster 
-------------------------------------------------------


 MySQL  localhost:33060+ ssl  JS > cluster.addInstance('icadmin@bosprod02:3306');

NOTE: The target instance 'bosprod02:3306' has not been pre-provisioned (GTID set is empty). 
The Shell is unable to decide whether incremental state recovery can correctly provision it.
The safest and most convenient way to provision a new instance is through automatic clone provisioning, 
which will completely overwrite the state of 'bosprod02:3306' with a physical snapshot from an existing cluster member. 
To use this method by default, set the 'recoveryMethod' option to 'clone'.

The incremental state recovery may be safely used if you are sure all updates ever executed in the cluster were 
done with GTIDs enabled, there are no purged transactions and the new instance contains the same GTID set as the 
cluster or a subset of it. To use this method by default, set the 'recoveryMethod' option to 'incremental'.

Please select a recovery method [C]lone/[I]ncremental recovery/[A]bort (default Clone): C
Validating instance configuration at bosprod02:3306...

This instance reports its own address as bosprod02:3306

Instance configuration is suitable.
NOTE: Group Replication will communicate with other members using 'bosprod02:3306'. Use the localAddress option to override.

* Checking connectivity and SSL configuration...

A new instance will be added to the InnoDB Cluster. Depending on the amount of
data on the cluster this might take from a few seconds to several hours.

Adding instance to the cluster...

Monitoring recovery process of the new cluster member. Press ^C to stop monitoring and let it continue in background.
Clone based state recovery is now in progress.

NOTE: A server restart is expected to happen as part of the clone process. If the
server does not support the RESTART command or does not come back after a
while, you may need to manually start it back.

* Waiting for clone to finish...
NOTE: bosprod02:3306 is being cloned from bosprod01:3306
** Stage DROP DATA: Completed 
** Clone Transfer  
    FILE COPY  ############################################################  100%  Completed
    PAGE COPY  ############################################################  100%  Completed
    REDO COPY  ############################################################  100%  Completed

NOTE: bosprod02:3306 is shutting down...

* Waiting for server restart... ready 
* bosprod02:3306 has restarted, waiting for clone to finish...
** Stage RESTART: Completed
* Clone process has finished: 72.60 MB transferred in about 1 second (~72.60 MB/s)

Incremental state recovery is now in progress.

* Waiting for distributed recovery to finish...
NOTE: 'bosprod02:3306' is being recovered from 'bosprod01:3306'
* Distributed recovery has finished

The instance 'bosprod02:3306' was successfully added to the cluster.



 MySQL  localhost:33060+ ssl  JS > cluster.status();
{
    "clusterName": "bosprodcluster", 
    "defaultReplicaSet": {
        "name": "default", 
        "primary": "bosprod01:3306", 
        "ssl": "REQUIRED", 
        "status": "OK_NO_TOLERANCE", 
        "statusText": "Cluster is NOT tolerant to any failures.", 
        "topology": {
            "bosprod01:3306": {
                "address": "bosprod01:3306", 
                "memberRole": "PRIMARY", 
                "mode": "R/W", 
                "readReplicas": {}, 
                "replicationLag": "applier_queue_applied", 
                "role": "HA", 
                "status": "ONLINE", 
                "version": "8.0.30"
            }, 
            "bosprod02:3306": {
                "address": "bosprod02:3306", 
                "memberRole": "SECONDARY", 
                "mode": "R/O", 
                "readReplicas": {}, 
                "replicationLag": "applier_queue_applied", 
                "role": "HA", 
                "status": "ONLINE", 
                "version": "8.0.30"
            }
        }, 
        "topologyMode": "Single-Primary"
    }, 
    "groupInformationSourceMember": "bosprod01:3306"
}
 MySQL  localhost:33060+ ssl  JS > 



MySQL  localhost:33060+ ssl  JS > cluster.addInstance('icadmin@bosprod03:3306');


Please select a recovery method [C]lone/[I]ncremental recovery/[A]bort (default Clone): C
Validating instance configuration at bosprod03:3306...

This instance reports its own address as bosprod03:3306

Instance configuration is suitable.
NOTE: Group Replication will communicate with other members using 'bosprod03:3306'. Use the localAddress option to override.

* Checking connectivity and SSL configuration...

A new instance will be added to the InnoDB Cluster. Depending on the amount of
data on the cluster this might take from a few seconds to several hours.

Adding instance to the cluster...

Monitoring recovery process of the new cluster member. Press ^C to stop monitoring and let it continue in background.
Clone based state recovery is now in progress.

NOTE: A server restart is expected to happen as part of the clone process. If the
server does not support the RESTART command or does not come back after a
while, you may need to manually start it back.

* Waiting for clone to finish...
NOTE: bosprod03:3306 is being cloned from bosprod02:3306
** Stage DROP DATA: Completed
** Clone Transfer  
    FILE COPY  ############################################################  100%  Completed
    PAGE COPY  ############################################################  100%  Completed
    REDO COPY  ############################################################  100%  Completed

NOTE: bosprod03:3306 is shutting down...

* Waiting for server restart... ready 
* bosprod03:3306 has restarted, waiting for clone to finish...
** Stage RESTART: Completed
* Clone process has finished: 72.61 MB transferred in about 1 second (~72.61 MB/s)

Incremental state recovery is now in progress.

* Waiting for distributed recovery to finish...
NOTE: 'bosprod03:3306' is being recovered from 'bosprod02:3306'
* Distributed recovery has finished

The instance 'bosprod03:3306' was successfully added to the cluster.



 MySQL  localhost:33060+ ssl  JS > cluster.status();
{
    "clusterName": "bosprodcluster", 
    "defaultReplicaSet": {
        "name": "default", 
        "primary": "bosprod01:3306", 
        "ssl": "REQUIRED", 
        "status": "OK", 
        "statusText": "Cluster is ONLINE and can tolerate up to ONE failure.", 
        "topology": {
            "bosprod01:3306": {
                "address": "bosprod01:3306", 
                "memberRole": "PRIMARY", 
                "mode": "R/W", 
                "readReplicas": {}, 
                "replicationLag": "applier_queue_applied", 
                "role": "HA", 
                "status": "ONLINE", 
                "version": "8.0.30"
            }, 
            "bosprod02:3306": {
                "address": "bosprod02:3306", 
                "memberRole": "SECONDARY", 
                "mode": "R/O", 
                "readReplicas": {}, 
                "replicationLag": "applier_queue_applied", 
                "role": "HA", 
                "status": "ONLINE", 
                "version": "8.0.30"
            }, 
            "bosprod03:3306": {
                "address": "bosprod03:3306", 
                "memberRole": "SECONDARY", 
                "mode": "R/O", 
                "readReplicas": {}, 
                "replicationLag": "applier_queue_applied", 
                "role": "HA", 
                "status": "ONLINE", 
                "version": "8.0.30"
            }
        }, 
        "topologyMode": "Single-Primary"
    }, 
    "groupInformationSourceMember": "bosprod01:3306"
}
 MySQL  localhost:33060+ ssl  JS > 


In [ ]:
Suppose all the mysql instance servers are down for a day and started 
=========================================================================

[root@bosprod02 ~]# mysqlsh

MySQL  localhost  SQL > \js
Switching to JavaScript mode...

MySQL  localhost  JS > \c icadmin@bosprod02
Please provide the password for 'icadmin@bosprod02': ************

MySQL  bosprod02:33060+ ssl  JS > cluster = dba.getCluster();
Dba.getCluster: This function is not available through a session to a standalone instance (metadata exists,
instance belongs to that metadata, but GR is not active) (MYSQLSH 51314)


Need to recreate cluster again 
-------------------------------------------

MySQL  bosprod02:33060+ ssl  JS > dba.createCluster('bosproddbcluster');

Dba.createCluster: dba.createCluster: Unable to create cluster. The instance 'bosprod02:3306' has a 
populated Metadata schema and belongs to that Metadata. Use either dba.dropMetadataSchema() to drop the schema, 
or dba.rebootClusterFromCompleteOutage() to reboot the cluster from complete outage. (RuntimeError)

MySQL  bosprod02:33060+ ssl  JS > dba.dropMetadataSchema();
Are you sure you want to remove the Metadata? [y/N]: y
Metadata Schema successfully removed.

MySQL  bosprod02:33060+ ssl  JS > dba.createCluster('bosproddbcluster');

NOTE: Group Replication will communicate with other members using 'bosprod02:3306'. Use the localAddress option to override.

* Checking connectivity and SSL configuration...

Creating InnoDB Cluster 'bosproddbcluster' on 'bosprod02:3306'...

Adding Seed Instance...

NOTE: User 'mysql_innodb_cluster_2'@'%' already existed at instance 'bosprod02:3306'. 
It will be deleted and created again with a new password.
Cluster successfully created. Use Cluster.addInstance() to add MySQL instances.
At least 3 instances are needed for the cluster to be able to withstand up to
one server failure.

 MySQL  bosprod02:33060+ ssl  JS > 


MySQL  bosprod02:33060+ ssl  JS > cluster = dba.getCluster();
<Cluster:bosproddbcluster>
 MySQL  bosprod02:33060+ ssl  JS > cluster.status();
{
    "clusterName": "bosproddbcluster", 
    "defaultReplicaSet": {
        "name": "default", 
        "primary": "bosprod02:3306", 
        "ssl": "REQUIRED", 
        "status": "OK_NO_TOLERANCE", 
        "statusText": "Cluster is NOT tolerant to any failures.", 
        "topology": {
            "bosprod02:3306": {
                "address": "bosprod02:3306", 
                "instanceErrors": [
                    "WARNING: Detected unused recovery accounts: mysql_innodb_cluster_1, mysql_innodb_cluster_3. 
                    Use Cluster.rescan() to clean up."
                ], 
                "memberRole": "PRIMARY", 
                "mode": "R/W", 
                "readReplicas": {}, 
                "replicationLag": "applier_queue_applied", 
                "role": "HA", 
                "status": "ONLINE", 
                "version": "8.0.30"
            }
        }, 
        "topologyMode": "Single-Primary"
    }, 
    "groupInformationSourceMember": "bosprod02:3306"
}
 MySQL  bosprod02:33060+ ssl  JS > 



Add instance bosprod01 to the cluster 
-----------------------------------------------


MySQL  bosprod02:33060+ ssl  JS > cluster.addInstance('icadmin@bosprod01');

Incremental state recovery was selected because it seems to be safely usable.

The instance 'bosprod01:3306' was successfully added to the cluster.

 

MySQL  bosprod02:33060+ ssl  JS > cluster.status();
{
    "clusterName": "bosproddbcluster", 
    "defaultReplicaSet": {
        "name": "default", 
        "primary": "bosprod02:3306", 
        "ssl": "REQUIRED", 
        "status": "OK_NO_TOLERANCE", 
        "statusText": "Cluster is NOT tolerant to any failures.", 
        "topology": {
            "bosprod01:3306": {
                "address": "bosprod01:3306", 
                "memberRole": "SECONDARY", 
                "mode": "R/O", 
                "readReplicas": {}, 
                "replicationLag": "applier_queue_applied", 
                "role": "HA", 
                "status": "ONLINE", 
                "version": "8.0.30"
            }, 
            "bosprod02:3306": {
                "address": "bosprod02:3306", 
                "instanceErrors": [
                    "WARNING: Detected an unused recovery account: mysql_innodb_cluster_3. Use Cluster.rescan() to clean up."
                ], 
                "memberRole": "PRIMARY", 
                "mode": "R/W", 
                "readReplicas": {}, 
                "replicationLag": "applier_queue_applied", 
                "role": "HA", 
                "status": "ONLINE", 
                "version": "8.0.30"
            }
        }, 
        "topologyMode": "Single-Primary"
    }, 
    "groupInformationSourceMember": "bosprod02:3306"
}
 MySQL  bosprod02:33060+ ssl  JS > 


Add instance bosprod03 to the cluster 
-----------------------------------------------

MySQL  bosprod02:33060+ ssl  JS > cluster.addInstance('icadmin@bosprod03');

Incremental state recovery was selected because it seems to be safely usable.

The instance 'bosprod03:3306' was successfully added to the cluster.

MySQL  bosprod02:33060+ ssl  JS > cluster.status();
{
    "clusterName": "bosproddbcluster", 
    "defaultReplicaSet": {
        "name": "default", 
        "primary": "bosprod02:3306", 
        "ssl": "REQUIRED", 
        "status": "OK", 
        "statusText": "Cluster is ONLINE and can tolerate up to ONE failure.", 
        "topology": {
            "bosprod01:3306": {
                "address": "bosprod01:3306", 
                "memberRole": "SECONDARY", 
                "mode": "R/O", 
                "readReplicas": {}, 
                "replicationLag": "applier_queue_applied", 
                "role": "HA", 
                "status": "ONLINE", 
                "version": "8.0.30"
            }, 
            "bosprod02:3306": {
                "address": "bosprod02:3306", 
                "memberRole": "PRIMARY", 
                "mode": "R/W", 
                "readReplicas": {}, 
                "replicationLag": "applier_queue_applied", 
                "role": "HA", 
                "status": "ONLINE", 
                "version": "8.0.30"
            }, 
            "bosprod03:3306": {
                "address": "bosprod03:3306", 
                "memberRole": "SECONDARY", 
                "mode": "R/O", 
                "readReplicas": {}, 
                "replicationLag": "applier_queue_applied", 
                "role": "HA", 
                "status": "ONLINE", 
                "version": "8.0.30"
            }
        }, 
        "topologyMode": "Single-Primary"
    }, 
    "groupInformationSourceMember": "bosprod02:3306"
}
 MySQL  bosprod02:33060+ ssl  JS > 


In [ ]:
Cluster commands for monitoring inndodb clsuter 
===============================================================


get cluster status 
--------------------------------

MySQL  localhost:33060+ ssl  JS > cluster = dba.getCluster();
<Cluster:bosproddbcluster>

 MySQL  localhost:33060+ ssl  JS > cluster.status();
{
    "clusterName": "bosproddbcluster", 
    "defaultReplicaSet": {
        "name": "default", 
        "primary": "bosprod02:3306", 
        "ssl": "REQUIRED", 
        "status": "OK", 
        "statusText": "Cluster is ONLINE and can tolerate up to ONE failure.", 
        "topology": {
            "bosprod01:3306": {
                "address": "bosprod01:3306", 
                "memberRole": "SECONDARY", 
                "mode": "R/O", 
                "readReplicas": {}, 
                "replicationLag": "applier_queue_applied", 
                "role": "HA", 
                "status": "ONLINE", 
                "version": "8.0.30"
            }, 
            "bosprod02:3306": {
                "address": "bosprod02:3306", 
                "memberRole": "PRIMARY", 
                "mode": "R/W", 
                "readReplicas": {}, 
                "replicationLag": "applier_queue_applied", 
                "role": "HA", 
                "status": "ONLINE", 
                "version": "8.0.30"
            }, 
            "bosprod03:3306": {
                "address": "bosprod03:3306", 
                "memberRole": "SECONDARY", 
                "mode": "R/O", 
                "readReplicas": {}, 
                "replicationLag": "applier_queue_applied", 
                "role": "HA", 
                "status": "ONLINE", 
                "version": "8.0.30"
            }
        }, 
        "topologyMode": "Single-Primary"
    }, 
    "groupInformationSourceMember": "bosprod02:3306"
}
 

describe the cluster 
---------------------------------

 MySQL  localhost:33060+ ssl  JS > cluster.describe();
{
    "clusterName": "bosproddbcluster", 
    "defaultReplicaSet": {
        "name": "default", 
        "topology": [
            {
                "address": "bosprod02:3306", 
                "label": "bosprod02:3306", 
                "role": "HA"
            }, 
            {
                "address": "bosprod01:3306", 
                "label": "bosprod01:3306", 
                "role": "HA"
            }, 
            {
                "address": "bosprod03:3306", 
                "label": "bosprod03:3306", 
                "role": "HA"
            }
        ], 
        "topologyMode": "Single-Primary"
    }
}
 MySQL  localhost:33060+ ssl  JS > 



Show all cluster commands 
-----------------------------------------------

 MySQL  localhost:33060+ ssl  JS > cluster.help();
NAME
      Cluster - Represents an InnoDB Cluster.

DESCRIPTION
      The cluster object is the entry point to manage and monitor a MySQL
      InnoDB Cluster.


show cluster name 
--------------------------

MySQL  localhost:33060+ ssl  JS > cluster.getName();
bosproddbcluster
 MySQL  localhost:33060+ ssl  JS > 

In [ ]:
Do some transactions in primary and check whether its reflected in all instances
===========================================================================================

mysql> select CHANNEL_NAME,MEMBER_HOST,MEMBER_STATE,MEMBER_ROLE
    -> from performance_schema.replication_group_members;
+---------------------------+-------------+--------------+-------------+
| CHANNEL_NAME              | MEMBER_HOST | MEMBER_STATE | MEMBER_ROLE |
+---------------------------+-------------+--------------+-------------+
| group_replication_applier | bosprod03   | ONLINE       | SECONDARY   |
| group_replication_applier | bosprod01   | ONLINE       | SECONDARY   |
| group_replication_applier | bosprod02   | ONLINE       | PRIMARY     |
+---------------------------+-------------+--------------+-------------+
3 rows in set (0.00 sec)

mysql> select CHANNEL_NAME,MEMBER_ID,COUNT_TRANSACTIONS_IN_QUEUE
    -> from performance_schema.replication_group_member_stats;
+---------------------------+--------------------------------------+-----------------------------+
| CHANNEL_NAME              | MEMBER_ID                            | COUNT_TRANSACTIONS_IN_QUEUE |
+---------------------------+--------------------------------------+-----------------------------+
| group_replication_applier | 40d021cc-2a41-11ef-af46-000c29009988 |                           0 |
| group_replication_applier | 85b5bb78-2a40-11ef-ae65-000c29051378 |                           0 |
| group_replication_applier | e5ecff5d-2a40-11ef-ae7f-000c293ea0bc |                           0 |
+---------------------------+--------------------------------------+-----------------------------+
3 rows in set (0.00 sec)


mysql> select @@hostname;
+------------+
| bosprod02  |
+------------+

mysql> create database bosp;
Query OK, 1 row affected (0.29 sec)

mysql> use bosp;
Database changed

mysql> create table orders (order_id int auto_increment primary key, order_details varchar(20) , order_date datetime default now() );
Query OK, 0 rows affected (0.22 sec)

mysql> delimiter //
mysql> create procedure sp_insert_records ()
    -> begin
    -> declare ord_count int default 100000;
    -> declare ord_init int default 1;
    -> while ( ord_init < ord_count ) do
    -> insert into orders (order_details) values ( concat('order details' , ord_init) );
    -> set ord_init = ord_init + 1;
    -> end while;
    -> end //
Query OK, 0 rows affected (0.06 sec)


mysql> call sp_insert_records();
Query OK, 1 row affected (7 min 45.89 sec)


mysql> select CHANNEL_NAME,MEMBER_ID,COUNT_TRANSACTIONS_CHECKED from performance_schema.replication_group_member_stats;
+---------------------------+--------------------------------------+----------------------------+
| CHANNEL_NAME              | MEMBER_ID                            | COUNT_TRANSACTIONS_CHECKED |
+---------------------------+--------------------------------------+----------------------------+
| group_replication_applier | 40d021cc-2a41-11ef-af46-000c29009988 |                      47465 |
| group_replication_applier | 85b5bb78-2a40-11ef-ae65-000c29051378 |                      47568 |
| group_replication_applier | e5ecff5d-2a40-11ef-ae7f-000c293ea0bc |                      47400 |
+---------------------------+--------------------------------------+----------------------------+
3 rows in set (0.00 sec)

mysql> select CHANNEL_NAME,MEMBER_ID,COUNT_TRANSACTIONS_CHECKED from performance_schema.replication_group_member_stats;
+---------------------------+--------------------------------------+----------------------------+
| CHANNEL_NAME              | MEMBER_ID                            | COUNT_TRANSACTIONS_CHECKED |
+---------------------------+--------------------------------------+----------------------------+
| group_replication_applier | 40d021cc-2a41-11ef-af46-000c29009988 |                     100006 |
| group_replication_applier | 85b5bb78-2a40-11ef-ae65-000c29051378 |                     100016 |
| group_replication_applier | e5ecff5d-2a40-11ef-ae7f-000c293ea0bc |                     100084 |
+---------------------------+--------------------------------------+----------------------------+
3 rows in set (0.00 sec)

mysql> select @@hostname , count(*) from orders;
+------------+----------+
| bosprod02  |    99999 |
+------------+----------+
+------------+----------+
| bosprod01  |    99999 |
+------------+----------+
+------------+----------+
| bosprod03  |    99999 |
+------------+----------+

In [ ]:
Switchover primary
==============================================

mysql> select CHANNEL_NAME,MEMBER_HOST,MEMBER_STATE,MEMBER_ROLE
    -> from performance_schema.replication_group_members;
+---------------------------+-------------+--------------+-------------+
| CHANNEL_NAME              | MEMBER_HOST | MEMBER_STATE | MEMBER_ROLE |
+---------------------------+-------------+--------------+-------------+
| group_replication_applier | bosprod03   | ONLINE       | SECONDARY   |
| group_replication_applier | bosprod01   | ONLINE       | SECONDARY   |
| group_replication_applier | bosprod02   | ONLINE       | PRIMARY     |
+---------------------------+-------------+--------------+-------------+


MySQL  bosprod02:33060+ ssl  JS > cluster.setPrimaryInstance("bosprod01");

Setting instance 'bosprod01' as the primary instance of cluster 'bosproddbcluster'...

Instance 'bosprod03:3306' remains SECONDARY.
Instance 'bosprod01:3306' was switched from SECONDARY to PRIMARY.
Instance 'bosprod02:3306' was switched from PRIMARY to SECONDARY.

The instance 'bosprod01' was successfully elected as primary.
MySQL  bosprod02:33060+ ssl  JS > 


mysql> select CHANNEL_NAME,MEMBER_HOST,MEMBER_STATE,MEMBER_ROLE
    -> from performance_schema.replication_group_members;
+---------------------------+-------------+--------------+-------------+
| CHANNEL_NAME              | MEMBER_HOST | MEMBER_STATE | MEMBER_ROLE |
+---------------------------+-------------+--------------+-------------+
| group_replication_applier | bosprod03   | ONLINE       | SECONDARY   |
| group_replication_applier | bosprod01   | ONLINE       | PRIMARY     |
| group_replication_applier | bosprod02   | ONLINE       | SECONDARY   |
+---------------------------+-------------+--------------+-------------+
3 rows in set (0.00 sec)

mysql> select @@hostname;
+------------+
| bosprod02  |
+------------+

mysql> call sp_insert_records();
ERROR 1290 (HY000): The MySQL server is running with the 
--super-read-only option so it cannot execute this statement
mysql> 

mysql> select @@hostname;
+------------+
| @@hostname |
+------------+
| bosprod01  |
+------------+
1 row in set (0.00 sec)

mysql> call sp_insert_records();
Query OK, 0 rows affected (6.82 sec)

mysql> select @@hostname , count(*) from orders;
+------------+----------+
| @@hostname | count(*) |
+------------+----------+
| bosprod01  |   199998 |
+------------+----------+
+------------+----------+
| bosprod02  |   199998 |
+------------+----------+
+------------+----------+
| bosprod03  |   199998 |
+------------+----------+


In [ ]:
Unexpected shutdown of primary server
===========================================================


[root@bosprod01 ~]# systemctl stop mysqld
[root@bosprod01 ~]# 


mysql> select CHANNEL_NAME,MEMBER_HOST,MEMBER_STATE,MEMBER_ROLE from performance_schema.replication_group_members;
+---------------------------+-------------+--------------+-------------+
| CHANNEL_NAME              | MEMBER_HOST | MEMBER_STATE | MEMBER_ROLE |
+---------------------------+-------------+--------------+-------------+
| group_replication_applier | bosprod03   | ONLINE       | PRIMARY     |
| group_replication_applier | bosprod02   | ONLINE       | SECONDARY   |
+---------------------------+-------------+--------------+-------------+


mysql> call sp_insert_records();
mysql> call sp_insert_records();
mysql> call sp_insert_records();

mysql> select @@hostname , count(*) from orders;
+------------+----------+
| @@hostname | count(*) |
+------------+----------+
| bosprod03  |   499995 |
+------------+----------+


primary server came up , join the instance to the cluster 
-------------------------------------------------------------------


MySQL  bosprod03:33060+ ssl  JS > cluster.addInstance("bosprod01");

The instance 'bosprod01:3306' was successfully added to the cluster.

MySQL  bosprod03:33060+ ssl  JS > 


mysql> select CHANNEL_NAME,MEMBER_HOST,MEMBER_STATE,MEMBER_ROLE from performance_schema.replication_group_members;
+---------------------------+-------------+--------------+-------------+
| CHANNEL_NAME              | MEMBER_HOST | MEMBER_STATE | MEMBER_ROLE |
+---------------------------+-------------+--------------+-------------+
| group_replication_applier | bosprod03   | ONLINE       | PRIMARY     |
| group_replication_applier | bosprod01   | ONLINE       | SECONDARY   |
| group_replication_applier | bosprod02   | ONLINE       | SECONDARY   |
+---------------------------+-------------+--------------+-------------+

mysql> select @@hostname , count(*) from orders;
+------------+----------+
| @@hostname | count(*) |
+------------+----------+
| bosprod01  |   499995 |
+------------+----------+


However the secondary instance bosprod02 is out of sync ( maybe checking with stale connection , 
should have connected with new connection to bosprod02 before below steps )
---------------------------------------------------------------------------------------------

mysql> select @@hostname , count(*) from orders;
+------------+----------+
| @@hostname | count(*) |
+------------+----------+
| bosprod02  |   199998 |
+------------+----------+

 MySQL  bosprod03:33060+ ssl  JS > cluster.removeInstance("bosprod02");

The instance 'bosprod02:3306' was successfully removed from the cluster.


MySQL  bosprod03:33060+ ssl  JS > cluster.rejoinInstance("bosprod02");
   Cluster.rejoinInstance: The instance 'bosprod02:3306' does not belong to the cluster: 
                           'bosproddbcluster'. (RuntimeError)


MySQL  bosprod03:33060+ ssl  JS > cluster.addInstance("bosprod02");

The instance 'bosprod02:3306' was successfully added to the cluster.


mysql> select @@hostname , count(*) from orders;
+------------+----------+
| @@hostname | count(*) |
+------------+----------+
| bosprod02  |   499995 |
+------------+----------+

In [ ]:
Switching cluster to multi-primary mode and vice-versa 
============================================================

mysql> select CHANNEL_NAME,MEMBER_HOST,MEMBER_STATE,MEMBER_ROLE from 
       performance_schema.replication_group_members;
+---------------------------+-------------+--------------+-------------+
| CHANNEL_NAME              | MEMBER_HOST | MEMBER_STATE | MEMBER_ROLE |
+---------------------------+-------------+--------------+-------------+
| group_replication_applier | bosprod03   | ONLINE       | PRIMARY     |
| group_replication_applier | bosprod01   | ONLINE       | SECONDARY   |
| group_replication_applier | bosprod02   | ONLINE       | SECONDARY   |
+---------------------------+-------------+--------------+-------------+


MySQL  bosprod03:33060+ ssl  JS > cluster.switchToMultiPrimaryMode();

Switching cluster 'bosproddbcluster' to Multi-Primary mode...

Instance 'bosprod03:3306' remains PRIMARY.
Instance 'bosprod01:3306' was switched from SECONDARY to PRIMARY.
Instance 'bosprod02:3306' was switched from SECONDARY to PRIMARY.

The cluster successfully switched to Multi-Primary mode.

MySQL  bosprod03:33060+ ssl  JS > 

+---------------------------+-------------+--------------+-------------+
| CHANNEL_NAME              | MEMBER_HOST | MEMBER_STATE | MEMBER_ROLE |
+---------------------------+-------------+--------------+-------------+
| group_replication_applier | bosprod03   | ONLINE       | PRIMARY     |
| group_replication_applier | bosprod01   | ONLINE       | PRIMARY     |
| group_replication_applier | bosprod02   | ONLINE       | PRIMARY     |
+---------------------------+-------------+--------------+-------------+


MySQL  bosprod03:33060+ ssl  JS > cluster.switchToSinglePrimaryMode();

Switching cluster 'bosproddbcluster' to Single-Primary mode...

Instance 'bosprod03:3306' remains PRIMARY.
Instance 'bosprod01:3306' was switched from PRIMARY to SECONDARY.
Instance 'bosprod02:3306' was switched from PRIMARY to SECONDARY.

WARNING: Existing connections that expected a R/W connection must be disconnected, i.e. instances that became SECONDARY.

The cluster successfully switched to Single-Primary mode.

MySQL  bosprod03:33060+ ssl  JS > 


+---------------------------+-------------+--------------+-------------+
| CHANNEL_NAME              | MEMBER_HOST | MEMBER_STATE | MEMBER_ROLE |
+---------------------------+-------------+--------------+-------------+
| group_replication_applier | bosprod03   | ONLINE       | PRIMARY     |
| group_replication_applier | bosprod01   | ONLINE       | SECONDARY   |
| group_replication_applier | bosprod02   | ONLINE       | SECONDARY   |
+---------------------------+-------------+--------------+-------------+

In [ ]:
Rebooting a cluster from a major outage 
=====================================================

if a cluster experiences a complete outage we can configure it using dba.rebootClutserFromCompleteOutage().

A complete outage means that group replication has stopped on all member instances.


 MySQL  localhost  JS > cluster = dba.getCluster();
Dba.getCluster: This function is not available through a session to a standalone instance 
(metadata exists, instance belongs to that metadata, but GR is not active) (MYSQLSH 51314)
 MySQL  localhost  JS > 

 
 mysql> select CHANNEL_NAME,MEMBER_HOST,MEMBER_STATE,MEMBER_ROLE from
    -> performance_schema.replication_group_members;
+---------------------------+-------------+--------------+-------------+
| CHANNEL_NAME              | MEMBER_HOST | MEMBER_STATE | MEMBER_ROLE |
+---------------------------+-------------+--------------+-------------+
| group_replication_applier | bosprod01   | OFFLINE      |             |
+---------------------------+-------------+--------------+-------------+


 MySQL  localhost  JS > \c icadmin@bosprod01
Creating a session to 'icadmin@bosprod01'
Please provide the password for 'icadmin@bosprod01': ************

 MySQL  bosprod01:33060+ ssl  JS > dba.rebootClusterFromCompleteOutage();
 
Restoring the Cluster 'bosproddbcluster' from complete outage...

Cluster instances: 'bosprod01:3306' (OFFLINE), 'bosprod02:3306' (OFFLINE), 'bosprod03:3306' (OFFLINE)

bosprod01:3306 was restored.
Updating instance metadata...
The instance metadata for 'bosprod01:3306' was successfully updated.

Rejoining instance 'bosprod03:3306' to cluster 'bosproddbcluster'...

* Distributed recovery has finished

The instance 'bosprod03:3306' was successfully rejoined to the cluster.

Rejoining instance 'bosprod02:3306' to cluster 'bosproddbcluster'...

* Distributed recovery has finished

The instance 'bosprod02:3306' was successfully rejoined to the cluster.

The Cluster was successfully rebooted.

<Cluster:bosproddbcluster>
 

+---------------------------+-------------+--------------+-------------+
| CHANNEL_NAME              | MEMBER_HOST | MEMBER_STATE | MEMBER_ROLE |
+---------------------------+-------------+--------------+-------------+
| group_replication_applier | bosprod03   | ONLINE       | SECONDARY   |
| group_replication_applier | bosprod01   | ONLINE       | PRIMARY     |
| group_replication_applier | bosprod02   | ONLINE       | SECONDARY   |
+---------------------------+-------------+--------------+-------------+

In [ ]:
Install Mysql Router 
=======================================

MySQL Router is part of InnoDB Cluster and is lightweight middleware that provides transparent routing between application 
and back-end MySQL Servers. 
It is used for a wide variety of use cases, such as providing high availability and scalability by routing database 
traffic to appropriate back-end MySQL servers.


[root@bosprod01 ~]# yum install mysql-router-community

Installed:
  mysql-router-community-8.4.0-1.el8.x86_64                                                                               

Complete!
[root@bosprod01 ~]# 


[root@bosprod01 ~]# mkdir -p /var/lib/mysqlrouter
[root@bosprod01 ~]# 
[root@bosprod01 ~]# chown -R mysql:mysql /var/lib/mysqlrouter
[root@bosprod01 ~]# 


[root@bosprod01 ~]# mysqlrouter --bootstrap localhost:3306 --directory /var/lib/mysqlrouter --user mysql
Please enter MySQL password for root: 

# Bootstrapping MySQL Router 8.4.0 (MySQL Community - GPL) instance at '/var/lib/mysqlrouter'...

- Creating account(s) (only those that are needed, if any)
- Verifying account (using it to run SQL queries that would be run by Router)
- Storing account in keyring
- Adjusting permissions of generated files
- Creating configuration /var/lib/mysqlrouter/mysqlrouter.conf

# MySQL Router configured for the InnoDB Cluster 'bosproddbcluster'

After this MySQL Router has been started with the generated configuration

    $ mysqlrouter -c /var/lib/mysqlrouter/mysqlrouter.conf

InnoDB Cluster 'bosproddbcluster' can be reached by connecting to:

## MySQL Classic protocol

- Read/Write Connections: localhost:6446
- Read/Only Connections:  localhost:6447
- Read/Write Split Connections: localhost:6450

## MySQL X protocol

- Read/Write Connections: localhost:6448
- Read/Only Connections:  localhost:6449

[root@bosprod01 ~]# 